In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
torch.cuda.set_device(0)

In [4]:
PATH = "/home/embsysa/fastai/pytorch_classifiers-master/data/"
sz = 224
arch = resnet50
bs = 24

## Initial exploration

In [5]:
import os
data_path = "/home/embsysa/fastai/pytorch_classifiers-master/data"
os.chdir(data_path)

In [6]:
!ls {PATH}

label_csv_name = 'Design_color_all_Multi_lables.csv'
label_df = pd.read_csv(label_csv_name)


 all_mask.zip				     Graphics_Multi_lables.csv.csv
 Background_color_allmultilabel.csv	     labels.zip
 Background_color_allmultilabel.csv.csv      Major_Background_labels.csv
 Background_color_all_Multi_lables.csv	     Major_Graphics_labels.csv
 Background_color_all_Multi_lables.csv.csv   Major_Graphics_labels.csv.csv
 Backup					     Major_Graphics_labels_final.csv
'change file type.ipynb'		     models
 cifar-10-batches-py			     old_Major_Background_labels.csv
 cifar-10-python.tar.gz			     test
 Design_color_all_Multi_lables.csv	     Theme_labels.csv
 Design_color_all_Multi_lables.csv.csv	     Theme_labels.csv.csv
 Design_contrast_color_labels.csv	     tmp
 Emotional_strength_labels.csv		     train
 Emotional_strength_labels.csv.csv	     train_py
 Emotion_labels.csv			     train_py_contrast
 Emotion_labels.csv.csv			     train_py_emotiona_srengh
'Final data Folders'			     train_py_major_graphics
'Final data Folders.zip'		     train_py_theme
 Graphics_Multi_lables.csv		     train_

In [7]:
images = [each.split(".jpg")[0] for each in os.listdir(data_path+"/train")]

In [8]:
label_df.head()

,Image_index,variable
0,28,4 6
1,40,1 3 4 8
2,698,1 2 4 5 8 9
3,769,1 2 3
4,786,1 6


In [9]:
label_df = label_df[label_df.Image_index.isin(images)]
label_df.to_csv(label_csv_name+'.csv', index=False)

In [10]:
label_csv_name

'Design_color_all_Multi_lables.csv'

In [11]:
label_csv = f'{PATH}'+label_csv_name+'.csv'
n = len(label_df)# header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [12]:
label_df.columns = label_df.columns.str.replace("-","_")
label_df.head()
#label_df.columns = ['id', 'color']

label_df.shape

(9955, 2)

In [13]:
#label_df.pivot_table(index="color", aggfunc=len).sort_values('id', ascending=False)
import numpy as np
np.max(val_idxs)

9954

In [14]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.2)
data = ImageClassifierData.from_csv(PATH, 'train', label_csv, test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

fn = PATH + data.trn_ds.fnames[0];
print(fn)

data.trn_ds.fnames[0]

img = PIL.Image.open(fn); img

img.size

/home/embsysa/fastai/pytorch_classifiers-master/data/train/40.jpg


(512, 512)

In [15]:
len(data.val_y)

1991

In [16]:
size_d = {k: PIL.Image.open(PATH + k).size for k in data.trn_ds.fnames}

In [17]:
row_sz, col_sz = list(zip(*size_d.values()))

In [18]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

## Initial model

In [19]:
def get_data(sz, bs): # sz: image size, bs: batch size
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.2)
    data = ImageClassifierData.from_csv(PATH, 'train', label_csv, test_name='test',
                                       val_idxs=val_idxs, suffix='.jpg', tfms=tfms,bs = bs)
    
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/13
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/37
    return data if sz > 300 else data.resize(256, 'tmp') # Reading the jpgs and resizing is slow for big images, so resizing them all to 340 first saves time

#Source:   
#    def resize(self, targ, new_path):
#        new_ds = []
#        dls = [self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl]
#        if self.test_dl: dls += [self.test_dl, self.test_aug_dl]
#        else: dls += [None,None]
#        t = tqdm_notebook(dls)
#        for dl in t: new_ds.append(self.resized(dl, targ, new_path))
#        t.close()
#        return self.__class__(new_ds[0].path, new_ds, self.bs, self.num_workers, self.classes)
#File:      ~/fastai/courses/dl1/fastai/dataset.py

In [20]:
from fastai.imports import *
from fastai.transforms import *
from fastai.dataset import *
from sklearn.metrics import fbeta_score
import warnings

def f2(preds, targs, start=0.17, end=0.24, step=0.01):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return max([fbeta_score(targs, (preds>th), 2, average='samples')
                    for th in np.arange(start,end,step)])

metrics=[f2]
f_model = resnet34

### Precompute

In [21]:
data = get_data(sz, bs)

In [22]:
learn = ConvLearner.pretrained(arch, data, precompute=False, metrics = metrics)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/embsysa/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:03<00:00, 26801743.18it/s]


In [23]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [0.01, 0.01, 0.1]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.


learn.precompute = False

In [25]:
learn.fit(1e-3, 10)

epoch      trn_loss   val_loss   f2                          
    0      0.153795   0.148198   0.746664  
    1      0.153146   0.147729   0.748073                    
    2      0.151768   0.147507   0.747969                    
    3      0.150855   0.14698    0.749022                    
    4      0.153408   0.146973   0.748813                    
    5      0.150437   0.146483   0.749338                    
    6      0.149901   0.146675   0.748847                    
    7      0.149115   0.146539   0.749169                    
    8      0.147235   0.145833   0.749854                    
    9      0.149241   0.14627    0.749073                    


[0.14626997643412806, 0.7490732790863766]

In [ ]:
learn.save('Multi_lables_Design_color')

### Validating the result 

In [ ]:
log_preds, y = learn.TTA() # (5, 2044, 120), (2044,)
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrics.log_loss(y, probs)

In [ ]:
y[0]

In [ ]:
f2(probs,y)

In [ ]:
probs[0]

### Augment

In [ ]:
from sklearn import metrics

In [ ]:
data = get_data(sz, bs)

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [ ]:
learn.fit(1e-2, 100)

In [ ]:
learn.precompute = False

In [ ]:
learn.fit(1e-2, 30, cycle_len=1)

In [ ]:
learn.save('Multi_lables_Design_color3')

In [ ]:
learn.load('Multi_lables_Design_color3')

## Increase size

In [ ]:
# Starting training on small images for a few epochs, then switching to bigger images, and continuing training is an amazingly effective way to avoid overfitting.

# http://forums.fast.ai/t/planet-classification-challenge/7824/96
# set_data doesn’t change the model at all. It just gives it new data to train with.
learn.set_data(get_data(299, bs)) 
learn.freeze()

#Source:   
#    def set_data(self, data, precompute=False):
#        super().set_data(data)
#        if precompute:
#            self.unfreeze()
#            self.save_fc1()
#            self.freeze()
#            self.precompute = True
#        else:
#            self.freeze()
#File:      ~/fastai/courses/dl1/fastai/conv_learner.py

In [ ]:
learn.summary()

In [ ]:
learn.fit(1e-2, 50, cycle_len=1)

Validation loss is much lower than training loss. This is a sign of underfitting. Cycle_len=1 may be too short. Let's set cycle_mult=2 to find better parameter.

In [ ]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [1e-5, 1e-5, 1e-4]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.

In [ ]:
# When you are under fitting, it means cycle_len=1 is too short (learning rate is getting reset before it had the chance to zoom in properly).
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2) # 1+2+4 = 7 epochs

Training loss and validation loss are getting closer and smaller. We are on right track.

In [ ]:
log_preds, y = learn.TTA() # (5, 2044, 120), (2044,)
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrics.log_loss(y, probs)

In [ ]:
label_df.iloc[3145,:]

In [ ]:
num = 23

print(val_idxs[num])
print(np.argmax(y[num]))
y[num]

In [ ]:
np.argmax(probs[num])

In [ ]:
probs[num]

In [ ]:
len(data.val_ds.y), data.val_ds.y[:5]

In [ ]:
num = 13
print(data.val_ds.y[num])
print(np.argmax(probs[num]))

print(probs[num])
data.val_ds.fnames[num]

In [ ]:
np.argmax(probs,axis = 1)

In [ ]:
predictions = pd.DataFrame([data.val_ds.y,np.argmax(probs,axis = 1),
                            [each[1] for each in np.argsort(-probs,axis = 1)],
                            data.val_ds.fnames]).T

In [ ]:
np.argmax(probs[1])

In [ ]:
print(probs[0])

In [ ]:
predictions.loc[:,'acc1'] = predictions[0] == predictions[1]
predictions.loc[:,'acc2'] = predictions[0] == predictions[2]
((predictions.acc1) | (predictions.acc2)).sum()

In [ ]:
len(predictions)

In [ ]:
predictions.acc1.sum()/1883

In [ ]:
learn.save('Multi_lables_Design_color1')

In [ ]:
learn.load('Multi_lables_Design_color2')

In [ ]:
learn.fit(1e-2, 1, cycle_len=2) # 1+1 = 2 epochs

In [ ]:
learn.save('Multi_lables_Design_color2')

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

This dataset is so similar to ImageNet dataset. Training convolution layers doesn't help much. We are not going to unfreeze.

## Create submission

https://youtu.be/9C06ZPF8Uuc?t=1905

In [ ]:
data.classes

In [ ]:
data.test_ds.fnames

In [ ]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrcs.log_loss(y, probs) # This does not make sense since test dataset has no labels

In [ ]:
probs.shape # (n_images, n_classes)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [ ]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]
fn

In [ ]:
Image.open(PATH + fn).resize((150, 150))

In [ ]:
# Method 1.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
learn.data.classes[np.argmax(preds)]

In [ ]:
# Method 2.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
im = val_tfms(open_image(PATH + fn)) # open_image() returns numpy.ndarray
preds = learn.predict_array(im[None])
np.argmax(preds)